In [21]:
import requests
import numpy as np
import tweepy

In [22]:
from keras.models import Sequential
from keras.layers import Dense
from textblob import TextBlob

In [23]:
consumer_key = "WZrFdXplH5lg5qsEglwnKhh3j"
comsumer_secret = "01iXX5V8tp5nNmOozuaF6qpgE5SA97W22bxONvVlRMbMuWKLFz"
access_token = "817989761419853825-SjetiEjTZGVPXi07QlL3VgC4W4RsHS4"
access_secret = "VsPCvnvyGWGu68eT8zJ9Nc67461w3Qq6pIC3TzyTjdZfS"
login = tweepy.OAuthHandler(consumer_key, comsumer_secret)
login.set_access_token(access_token, access_secret)
user = tweepy.API(login)

file = 'historical.csv'

In [24]:
def get_name(symbol):
    url = "http://d.yimg.com/autoc.finance.yahoo.com/autoc?query={}&region=1&lang=en".format(symbol)
    result = requests.get(url).json()
    for x in result['ResultSet']['Result']:
        if x['symbol'] == symbol:
            return x['name']

In [25]:
def sentiment(quote, num):
    tweet_list = user.search(get_name(quote), count = num)
    positive = 0
    null = 0
    for tweet in tweet_list:
        check = TextBlob(tweet.text).sentiment
        if check.subjectivity == 0:
            null += 1
            next
        if check.polarity > 0:
            positive += 1

    if positive > ((num - null)/2):
        return True

In [26]:
def get_data(quote):
    url = 'http://www.google.com/finance/historical?q=NASDAQ%3A'+quote+'&output=csv'
    r = requests.get(url, stream = True)

    if r.status_code != 400:
        with open(file, 'wb') as fl:
            for line in r:
                fl.write(line)
    return True

In [27]:
def predict():
    data = []
    with open(file) as f:
        for num, line in enumerate(f):
            if num != 0:
                data.append(float(line.split(',')[1]))
    data = np.array(data)

    def create_set(data):
        datax = [data[n+1] for n in range(len(data)-2)]
        return np.array(datax), data[2:]

    trainx, trainy = create_set(data)

    classifier = Sequential()
    classifier.add(Dense(8, input_dim = 1, activation = 'relu'))
    classifier.add(Dense(1))
    classifier.compile(loss = 'mean_squared_error', optimizer = 'adam')
    classifier.fit(trainx, trainy, epochs= 200, batch_size = 2, verbose = 2)

    prediction = classifier.predict(np.array([data[0]]))
    return 'from %s to %s' % (data[0], prediction[0][0])

quote = input('Enter stock quote: ').upper()


if not get_data(quote):
    print ('ERROR, please re-run the script')

print(predict())

if not sentiment(quote, num = 100):
    print ('This stock has bad sentiment')
else:
    print ('This stock has good sentiment')

Enter stock quote: GOOGL
Epoch 1/200
0s - loss: 1180803.4607
Epoch 2/200
0s - loss: 987848.5736
Epoch 3/200
0s - loss: 863165.6139
Epoch 4/200
0s - loss: 776470.0000
Epoch 5/200
0s - loss: 753548.2737
Epoch 6/200
0s - loss: 753350.8201
Epoch 7/200
0s - loss: 753150.0469
Epoch 8/200
0s - loss: 752946.7757
Epoch 9/200
0s - loss: 752741.5776
Epoch 10/200
0s - loss: 752534.9330
Epoch 11/200
0s - loss: 752327.2692
Epoch 12/200
0s - loss: 752118.6084
Epoch 13/200
0s - loss: 751909.2686
Epoch 14/200
0s - loss: 751699.3679
Epoch 15/200
0s - loss: 751488.8584
Epoch 16/200
0s - loss: 751277.9748
Epoch 17/200
0s - loss: 751066.7858
Epoch 18/200
0s - loss: 750855.2828
Epoch 19/200
0s - loss: 750643.5398
Epoch 20/200
0s - loss: 750431.6310
Epoch 21/200
0s - loss: 750219.4763
Epoch 22/200
0s - loss: 750007.1890
Epoch 23/200
0s - loss: 749794.8231
Epoch 24/200
0s - loss: 749582.3065
Epoch 25/200
0s - loss: 749369.7707
Epoch 26/200
0s - loss: 749157.0948
Epoch 27/200
0s - loss: 748944.4123
Epoch 28/20